In [13]:
import ee
import geemap.eefolium as geemap
import os
import requests
import math
import json
import datetime


Map = geemap.Map()
Map

In [14]:
def maskL8sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

In [15]:
#MASKING FUNCTION FOR L5 AND L7
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
    # If the cloud bit (5) is set and the cloud confidence (7) is high
    # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5) \
                  .And(qa.bitwiseAnd(1 << 7)) \
                  .Or(qa.bitwiseAnd(1 << 3))
    # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2)

In [16]:
def generateLandsatImage(year):
    if(year<1999):
        dataset_land = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
                      .map(cloudMaskL457) \

        image_land = dataset_land.select(['B3', 'B2', 'B1']).median()
        land_vis['bands'] = ['B3', 'B2', 'B1']
    elif(year<2013):
        dataset_land = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
                      .map(cloudMaskL457) \

        image_land = dataset_land.select(['B3', 'B2', 'B1']).median()
        land_vis['bands'] = ['B3', 'B2', 'B1']
    else:
        dataset_land = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
                      .map(maskL8sr) \

        image_land = dataset_land.select(['B4', 'B3', 'B2']).median()
        land_vis['bands'] = ["B4","B3","B2"]

    url_land = image_land.getThumbURL(land_vis)
    return url_land

In [17]:
def generateWaterMapImage(year):
    dataset_water = ee.ImageCollection("JRC/GSW1_2/YearlyHistory") \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31')
    image_water = dataset_water.select(['waterClass']).median()
    url_water = image_water.getThumbURL(water_vis)
    return url_water

In [18]:
def exportLocallyLand(URL, fileName):
    r = requests.get(URL, stream=True)
    out_png = os.path.expanduser('~/Desktop/picture_samples/land/'+fileName+'.png')

    if r.status_code != 200:
        print('An error occurred while downloading.')
    else:
        with open(out_png, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=1024):
                fd.write(chunk)
        print(fileName+ ' created!')

In [19]:
def exportLocallyWater(URL, fileName):
    r = requests.get(URL, stream=True)
    out_png = os.path.expanduser('~/Desktop/picture_samples/water/'+fileName+'.png')

    if r.status_code != 200:
        print('An error occurred while downloading.')
    else:
        with open(out_png, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=1024):
                fd.write(chunk)
        print(fileName+ ' created!')

In [20]:
#Used for pixel dimension consistency since distance will always be the same
def calculatePoint(latitude, longitude, bearing, distance):
    
    R = 6378.1 #Radius of the Earth
    brng = math.radians(bearing) #Bearing is 135
    d = distance #Distance in km


    lat1_rad = math.radians(latitude) #Current lat point converted to radians
    lon1_rad = math.radians(longitude) #Current long point converted to radians

    lat2_cal = math.asin( math.sin(lat1_rad)*math.cos(d/R) + math.cos(lat1_rad)*math.sin(d/R)*math.cos(brng))

    lon2_cal = lon1_rad + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1_rad),math.cos(d/R)-math.sin(lat1_rad)*math.sin(lat2_cal))

    lat2_deg = math.degrees(lat2_cal)
    lon2_deg = math.degrees(lon2_cal)

    return([lon2_deg,lat2_deg])

In [21]:
def generatePhotos(lat, lng, startY, endY, index, time):
    i = 0
        
    for year in range(startY,endY,1):
        fileName = str(lat) +'_'+ str(lng) +'_'+ str(year)
        landURL = generateLandsatImage(year)
        waterURL = generateWaterMapImage(year)

        i = i + 1
    
        time_elapsed = datetime.datetime.now() - datetime.timedelta(hours=time.hour, minutes=time.minute, seconds=time.second)
        print("Started Number:" + str(index) + "/150      Year:" + str(i) +"/33" "   Time Elapsed: " + str(time_elapsed.hour)+":" + str(time_elapsed.minute)+":"+str(time_elapsed.second))
        exportLocallyLand(landURL, fileName)
        exportLocallyWater(waterURL, fileName)

In [22]:
##CHANGE FOR NUMBER OF ITERATIONS
START_YEAR = 1997 # MAX = 1986, MIN = 2019 ----- MAX 1984 & 85 IS POSSIBLE BUT DATA USUALLY NOT THAT GOOD
END_YEAR = 2020 # should be 2020 always, change only for testing

In [23]:
start_time = datetime.datetime.now()
with open('../ReactJs/src/north_america_150.json') as json_file:
    data = json.load(json_file)
    i = 0
    for coords in data:
        
        pointA = calculatePoint(coords['latitude'], coords['longitude'], 135, 5)
        pointB = calculatePoint(coords['latitude'], coords['longitude'], 315, 5)
        
        #THIS SHOULD BE LATITUDE, LONGITUDE, LATITUDE, LONGITUDE
        #BUT FOR SOME BLOODY REASON ee TAKES THE REVERSE ????
        roi = ee.Geometry.Rectangle([pointA[0], pointA[1], pointB[0], pointB[1]])

        #IMAGE VISUALIZATION PARAMETERS
        land_vis = {
            'gamma': 1,
            'max': 3000,
            'min': 0,
            'scale': 5,
            'region': roi,
            'format': 'png'
            }

        water_vis = {
            'bands': ['waterClass'],
            'palette': ["cccccc","ffffff","99d9ea","0000ff"],
            'max': 3,
            'min': 0,
            'scale': 5,
            'region': roi,
            'format': 'png'
        }
        i = i + 1
        if(i > 130):
            generatePhotos(round(coords['latitude'],4), round(coords['longitude'],4), START_YEAR, END_YEAR, i, start_time)


Started Number:131/150      Year:1/33   Time Elapsed: 0:0:4
53.9317_-100.6172_1997 created!
53.9317_-100.6172_1997 created!
Started Number:131/150      Year:2/33   Time Elapsed: 0:0:59
53.9317_-100.6172_1998 created!
53.9317_-100.6172_1998 created!
Started Number:131/150      Year:3/33   Time Elapsed: 0:2:10
53.9317_-100.6172_1999 created!
53.9317_-100.6172_1999 created!
Started Number:131/150      Year:4/33   Time Elapsed: 0:3:10
53.9317_-100.6172_2000 created!
53.9317_-100.6172_2000 created!
Started Number:131/150      Year:5/33   Time Elapsed: 0:4:16
53.9317_-100.6172_2001 created!
53.9317_-100.6172_2001 created!
Started Number:131/150      Year:6/33   Time Elapsed: 0:6:1
53.9317_-100.6172_2002 created!
53.9317_-100.6172_2002 created!
Started Number:131/150      Year:7/33   Time Elapsed: 0:7:45
53.9317_-100.6172_2003 created!
53.9317_-100.6172_2003 created!
Started Number:131/150      Year:8/33   Time Elapsed: 0:8:29
53.9317_-100.6172_2004 created!
53.9317_-100.6172_2004 created!
St

38.2424_-103.266_2016 created!
38.2424_-103.266_2016 created!
Started Number:133/150      Year:21/33   Time Elapsed: 1:14:44
38.2424_-103.266_2017 created!
38.2424_-103.266_2017 created!
Started Number:133/150      Year:22/33   Time Elapsed: 1:15:34
38.2424_-103.266_2018 created!
38.2424_-103.266_2018 created!
Started Number:133/150      Year:23/33   Time Elapsed: 1:16:28
38.2424_-103.266_2019 created!
38.2424_-103.266_2019 created!
Started Number:134/150      Year:1/33   Time Elapsed: 1:17:16
38.2758_-102.6329_1997 created!
38.2758_-102.6329_1997 created!
Started Number:134/150      Year:2/33   Time Elapsed: 1:18:47
38.2758_-102.6329_1998 created!
38.2758_-102.6329_1998 created!
Started Number:134/150      Year:3/33   Time Elapsed: 1:20:50
38.2758_-102.6329_1999 created!
38.2758_-102.6329_1999 created!
Started Number:134/150      Year:4/33   Time Elapsed: 1:21:56
38.2758_-102.6329_2000 created!
38.2758_-102.6329_2000 created!
Started Number:134/150      Year:5/33   Time Elapsed: 1:23:

38.3501_-102.6988_2012 created!
38.3501_-102.6988_2012 created!
Started Number:136/150      Year:17/33   Time Elapsed: 2:46:52
38.3501_-102.6988_2013 created!
38.3501_-102.6988_2013 created!
Started Number:136/150      Year:18/33   Time Elapsed: 2:47:46
38.3501_-102.6988_2014 created!
38.3501_-102.6988_2014 created!
Started Number:136/150      Year:19/33   Time Elapsed: 2:48:45
38.3501_-102.6988_2015 created!
38.3501_-102.6988_2015 created!
Started Number:136/150      Year:20/33   Time Elapsed: 2:49:40
38.3501_-102.6988_2016 created!
38.3501_-102.6988_2016 created!
Started Number:136/150      Year:21/33   Time Elapsed: 2:50:41
38.3501_-102.6988_2017 created!
38.3501_-102.6988_2017 created!
Started Number:136/150      Year:22/33   Time Elapsed: 2:51:57
38.3501_-102.6988_2018 created!
38.3501_-102.6988_2018 created!
Started Number:136/150      Year:23/33   Time Elapsed: 2:53:28
38.3501_-102.6988_2019 created!
38.3501_-102.6988_2019 created!
Started Number:137/150      Year:1/33   Time El

38.2614_-104.7392_2008 created!
38.2614_-104.7392_2008 created!
Started Number:139/150      Year:13/33   Time Elapsed: 3:59:4
38.2614_-104.7392_2009 created!
38.2614_-104.7392_2009 created!
Started Number:139/150      Year:14/33   Time Elapsed: 4:0:4
38.2614_-104.7392_2010 created!
38.2614_-104.7392_2010 created!
Started Number:139/150      Year:15/33   Time Elapsed: 4:1:9
38.2614_-104.7392_2011 created!
38.2614_-104.7392_2011 created!
Started Number:139/150      Year:16/33   Time Elapsed: 4:2:19
38.2614_-104.7392_2012 created!
38.2614_-104.7392_2012 created!
Started Number:139/150      Year:17/33   Time Elapsed: 4:3:9
38.2614_-104.7392_2013 created!
38.2614_-104.7392_2013 created!
Started Number:139/150      Year:18/33   Time Elapsed: 4:3:45
38.2614_-104.7392_2014 created!
38.2614_-104.7392_2014 created!
Started Number:139/150      Year:19/33   Time Elapsed: 4:4:33
38.2614_-104.7392_2015 created!
38.2614_-104.7392_2015 created!
Started Number:139/150      Year:20/33   Time Elapsed: 4:

38.1697_-104.6513_2004 created!
38.1697_-104.6513_2004 created!
Started Number:142/150      Year:9/33   Time Elapsed: 5:4:53
38.1697_-104.6513_2005 created!
38.1697_-104.6513_2005 created!
Started Number:142/150      Year:10/33   Time Elapsed: 5:6:17
38.1697_-104.6513_2006 created!
38.1697_-104.6513_2006 created!
Started Number:142/150      Year:11/33   Time Elapsed: 5:7:15
38.1697_-104.6513_2007 created!
38.1697_-104.6513_2007 created!
Started Number:142/150      Year:12/33   Time Elapsed: 5:8:25
38.1697_-104.6513_2008 created!
38.1697_-104.6513_2008 created!
Started Number:142/150      Year:13/33   Time Elapsed: 5:9:25
38.1697_-104.6513_2009 created!
38.1697_-104.6513_2009 created!
Started Number:142/150      Year:14/33   Time Elapsed: 5:10:32
38.1697_-104.6513_2010 created!
38.1697_-104.6513_2010 created!
Started Number:142/150      Year:15/33   Time Elapsed: 5:11:32
38.1697_-104.6513_2011 created!
38.1697_-104.6513_2011 created!
Started Number:142/150      Year:16/33   Time Elapsed

44.694_-116.1116_2001 created!
Started Number:145/150      Year:6/33   Time Elapsed: 6:44:42
44.694_-116.1116_2002 created!
44.694_-116.1116_2002 created!
Started Number:145/150      Year:7/33   Time Elapsed: 6:45:50
44.694_-116.1116_2003 created!
44.694_-116.1116_2003 created!
Started Number:145/150      Year:8/33   Time Elapsed: 6:46:36
44.694_-116.1116_2004 created!
44.694_-116.1116_2004 created!
Started Number:145/150      Year:9/33   Time Elapsed: 6:47:54
44.694_-116.1116_2005 created!
44.694_-116.1116_2005 created!
Started Number:145/150      Year:10/33   Time Elapsed: 6:48:49
44.694_-116.1116_2006 created!
44.694_-116.1116_2006 created!
Started Number:145/150      Year:11/33   Time Elapsed: 6:49:47
44.694_-116.1116_2007 created!
44.694_-116.1116_2007 created!
Started Number:145/150      Year:12/33   Time Elapsed: 6:50:49
44.694_-116.1116_2008 created!
44.694_-116.1116_2008 created!
Started Number:145/150      Year:13/33   Time Elapsed: 6:51:47
44.694_-116.1116_2009 created!
44.6

44.5141_-116.4371_1998 created!
44.5141_-116.4371_1998 created!
Started Number:148/150      Year:3/33   Time Elapsed: 7:43:0
44.5141_-116.4371_1999 created!
44.5141_-116.4371_1999 created!
Started Number:148/150      Year:4/33   Time Elapsed: 7:43:44
44.5141_-116.4371_2000 created!
44.5141_-116.4371_2000 created!
Started Number:148/150      Year:5/33   Time Elapsed: 7:45:4
44.5141_-116.4371_2001 created!
44.5141_-116.4371_2001 created!
Started Number:148/150      Year:6/33   Time Elapsed: 7:46:34
44.5141_-116.4371_2002 created!
44.5141_-116.4371_2002 created!
Started Number:148/150      Year:7/33   Time Elapsed: 7:47:43
44.5141_-116.4371_2003 created!
44.5141_-116.4371_2003 created!
Started Number:148/150      Year:8/33   Time Elapsed: 7:48:27
44.5141_-116.4371_2004 created!
44.5141_-116.4371_2004 created!
Started Number:148/150      Year:9/33   Time Elapsed: 7:49:35
44.5141_-116.4371_2005 created!
44.5141_-116.4371_2005 created!
Started Number:148/150      Year:10/33   Time Elapsed: 7

44.564_-117.1436_2017 created!
Started Number:150/150      Year:22/33   Time Elapsed: 8:41:14
44.564_-117.1436_2018 created!
44.564_-117.1436_2018 created!
Started Number:150/150      Year:23/33   Time Elapsed: 8:42:2
44.564_-117.1436_2019 created!
44.564_-117.1436_2019 created!


In [24]:
#########################################################################################
# WHEN MAKING ANY CHANGES, MAKE SURE TO RUN ALL CELLS AND NOT JUST THE CELL YOU CHANGED #
#########################################################################################